In [23]:
library(rliger)
# library(liger)
library(Seurat)
library(Signac)
library(Matrix)
library(patchwork)
# library(EnsDb.Hsapiens.v86)
library(ggplot2)
library(tidyverse)
library(anndata)
library(SeuratDisk)

In [24]:
# setwd("~/Documents/xsede/scDART_full_test/test")
################################################

results_dir <- "/home/xcx/results/uterus/8-Liger"

counts.rna <- Matrix::readMM("/data1/xcx/unpaired-data/uterus/RNA/highvar_counts_rna.mtx")
counts.rna <- as.matrix(t(counts.rna))
rownames(counts.rna) <- paste("Gene-", seq(1, dim(counts.rna)[1]), sep = "")
colnames(counts.rna) <- paste("Cell-", seq(1, dim(counts.rna)[2]), sep = "")

counts.atac <- Matrix::readMM("/data1/xcx/unpaired-data/uterus/ATAC/uterus_Zhang2021/highvar_counts_atac.mtx")
counts.atac <- as.matrix(t(counts.atac))
rownames(counts.atac) <- paste("Region-", seq(1, dim(counts.atac)[1]), sep = "")
colnames(counts.atac) <- paste("Cell-", seq(dim(counts.rna)[2] + 1, dim(counts.rna)[2] + dim(counts.atac)[2]), sep = "")

region2gene <- Matrix::readMM("/data1/xcx/unpaired-data/uterus/highvar_region2gene.mtx")
region2gene <- as.matrix(region2gene)
rownames(region2gene) <- paste("Region-", seq(1, dim(region2gene)[1]), sep = "")
colnames(region2gene) <- paste("Gene-", seq(1, dim(region2gene)[2]), sep = "")

activity.matrix <- t(region2gene) %*% counts.atac

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.0 GiB”


In [ ]:
label.rna <- read_h5ad("/data1/xcx/unpaired-data/vessel/RNA/vessel_selected_intergrated.h5ad")
label.atac <- read_h5ad("/data1/xcx/unpaired-data/vessel/ATAC/vessel_Zhang2021/raw_h5ad/adata_selected_changedname.h5ad")

In [26]:
start_time <- proc.time()

ifnb_liger <- rliger::createLiger(list(rna1 = counts.rna, atac2 = activity.matrix), remove.missing = FALSE)
# ifnb_liger <- rliger::createLiger(list(rna1 = counts.rna, atac2 = counts.atac), remove.missing = FALSE)
ifnb_liger <- rliger::normalize(ifnb_liger, verbose = TRUE)
# select all the genes
ifnb_liger <- rliger::selectGenes(ifnb_liger, var.thresh = 1e-6)
ifnb_liger

# ifnb_liger@var.genes <- paste("Gene_", seq(1, dim(region2gene)[2]), sep = "")
ifnb_liger <- rliger::scaleNotCenter(ifnb_liger, remove.missing = FALSE)
ifnb_liger

Warning message in sqrt(gene_expr_mean * nolan_constant/ncol(object@raw.data[[i]])):
“NaNs produced”
Warning message in rliger::selectGenes(ifnb_liger, var.thresh = 1e-06):
“NaNs produced”


An object of class liger 
with 2 datasets and
 16551 total cells.

An object of class liger 
with 2 datasets and
 16551 total cells.

In [27]:
#use lambda 5 k 20
#integrate NMF
ifnb_liger <- optimizeALS(ifnb_liger,k = 4,lambda = 5, max.iters = 50,thresh = 1e-6, remove.missing = FALSE)

  |======================================================================| 100%
Finished in 2.84473 mins, 50 iterations.
Max iterations set: 50.
Final objective delta: 3.186747e-06.
Best results with seed 1.


In [28]:
#Quantile Normalization and Joint Clustering
ifnb_liger <- quantile_norm(ifnb_liger,knn_k = 20,quantiles = 50,min_cells = 20,do.center = FALSE,
                            max_sample = 1000,refine.knn = TRUE,eps = 0.9)
ifnb_liger

An object of class liger 
with 2 datasets and
 16551 total cells.

In [29]:
elapsed_time <- proc.time() - start_time
elapsed_time

    user   system  elapsed 
1044.613 3072.597  205.670 

In [30]:
# you can use louvain cluster to detect and assign cluster
ifnb_liger <- louvainCluster(ifnb_liger, resolution = 0.25)

Louvain Clustering on quantile normalized cell factor loadings.



In [31]:
# quantile normalization
print("Writing results...")

# ifnb_liger <- rliger::quantile_norm(ifnb_liger)  # 这里好像重复了
H1 <- ifnb_liger@H$rna1
H2 <- ifnb_liger@H$atac2

# ifnb_liger@H

H.norm1 <- ifnb_liger@H.norm[0:dim(H1)[1],]
H.norm2 <- ifnb_liger@H.norm[(dim(H1)[1] + 1):(dim(H1)[1] + dim(H2)[1]),]

H.norm1
H.norm2

[1] "Writing results..."


Cell-1,5.455630e-03,0.008508118,0.001806064,0.0020082069
Cell-2,0.000000e+00,0.005091679,0.002262963,0.0131784060
Cell-3,1.990443e-05,0.011696342,0.011699553,0.0038553538
Cell-4,6.231495e-03,0.004545877,0.001806064,0.0022256718
Cell-5,0.000000e+00,0.005091679,0.012116751,0.0088950373
Cell-6,1.279350e-03,0.007484959,0.017240464,0.0002736929
Cell-7,4.124170e-03,0.004545877,0.001806064,0.0029601975
Cell-8,3.124479e-03,0.014086801,0.006262738,0.0051586018
Cell-9,5.771097e-03,0.008867660,0.010813878,0.0000000000
Cell-10,0.000000e+00,0.005091679,0.014469344,0.0058025520
Cell-11,0.000000e+00,0.005091679,0.002262963,0.0102486792


Cell-8238,0.000000e+00,0.0132866074,1.510369e-02,0.001715109
Cell-8239,1.994989e-03,0.0221111259,0.000000e+00,0.000000000
Cell-8240,0.000000e+00,0.0038310475,1.421221e-02,0.008931371
Cell-8241,0.000000e+00,0.0082108632,1.928897e-02,0.004599808
Cell-8242,2.655488e-03,0.0079990430,1.273292e-02,0.003099351
Cell-8243,3.223763e-03,0.0040585420,1.841002e-02,0.001682965
Cell-8244,3.238426e-03,0.0126834837,0.000000e+00,0.004203658
Cell-8245,0.000000e+00,0.0051201606,2.428646e-02,0.001335715
Cell-8246,2.778701e-03,0.0072617444,1.766771e-02,0.000000000
Cell-8247,3.960693e-04,0.0102811835,1.412098e-02,0.004741630
Cell-8248,0.000000e+00,0.0004191644,3.553335e-02,0.005570936


In [32]:
write.csv(H.norm1, "/home/xcx/results/uterus/8-Liger/z_rna.csv")
write.csv(H.norm2, "/home/xcx/results/uterus/8-Liger/z_atac.csv")